# Destination Scoring
This notebook joins occupancy and rate data to compute a simple score for each destination.


In [ ]:
from pathlib import Path
import pandas as pd


In [ ]:
clean_dir = Path('Data/clean')
acc = pd.read_csv(clean_dir / 'accommodation_facts_clean.csv')
market = pd.read_csv(clean_dir / 'market_otb_clean.csv', parse_dates=['stay_date'])
rates = pd.read_csv(clean_dir / 'rates_combined.csv', parse_dates=['stay_date', 'extract_date'])

## Aggregate metrics by destination

In [ ]:
# Average rate per destination
rates_with_dest = rates.merge(acc[['bookingdotcom_id','travel_destination_name']], on='bookingdotcom_id', how='left')
dest_rate = rates_with_dest.groupby('travel_destination_name')['price_value'].mean()

In [ ]:
# Average occupancy per destination
dest_occ = market.groupby('travel_destination_name')['average_occupancy'].mean()

## Compute a combined score

In [ ]:
score = dest_rate.to_frame('avg_rate').join(dest_occ.to_frame('avg_occupancy'))
score['score'] = score['avg_rate'] * score['avg_occupancy']
score.sort_values('score', ascending=False).head()